## You.com x LangChain

### YouRetriever
Today we are excited to announce the release of `YouRetriever`, the easiest way to get access to the You.com Search API. The You.com Search API is designed by LLMs for LLMs with an emphasis on Retrieval Augmented Generation (RAG) applications. We accomplish this by evaluating our API on a number of datasets to benchmark performance of LLMs in the RAG-QA setting. In this blog post we will compare and contrast the You.com Search API with the Google Search API as well as give the reader the tools to evaluate LLMs in the RAG-QA setting. We will evaluate our retriever performance on [Hotpot QA](https://github.com/hotpotqa/hotpot) using the `RetrievalQA` Chain. Hotpot is a dataset which is comprised of a question, answer, and context. The context can vary in relevance to the question/answer with a special "distractor" setting where the LLM needs to not be distracted by certain misleading text within the context. In this experiment we will be removing the context from the dataset and replacing it with text snippets which come back from the search APIs. In this sense the entire internet is the distractor text since the APIs are responsible for finding the answer to the question across the entire internet not just within the list of snippets supplied in the dataset. We call this the "web distractor" setting for evaluating search APIs with respect to their performance being used in conjunction with an LLM.

In [1]:
pip install -q langchain==0.0.314 datasets google-api-python-client openai==0.27.4 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

### Initialization
All you need to initialize a `YouRetriever` is to set the environment variable `YDC_API_KEY`. We are currently in an alpha status and keys are available by invitation only. If you are interested in being an early access partner please email api@you.com with your usecase, background, and expected daily load. You will also need an OpenAI key along with credentials to the Google Search API to run the rest of the notebook.

In [ ]:
import os


os.environ["YDC_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""

In [4]:
%pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.5 MB/s eta 0:00:00


In [3]:
from langchain.retrievers.web_research import GoogleSearchAPIWrapper


yr = GoogleSearchAPIWrapper() #aYouRetriever()

ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass  `google_api_key` as a named parameter. (type=value_error)

In [7]:
%pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.314
    Uninstalling langchain-0.0.314:
      Successfully uninstalled langchain-0.0.314


In [15]:
from langchain_community.tools import DuckDuckGoSearchRun# Initialize the tool
yr = DuckDuckGoSearchRun()

### Retrieval
The first thing you will notice about our text snippets is that we provide larger text snippets when we can and will soon have the option for specifying the amount of text you want returned from a single snippet to the entire page. Let's ask it about the greatest pinball player ever, Keith Elwin

In [22]:

# Use the tool to search the internet
search_query  = "keith elwin pinball designer"
search_results = yr.run(  search_query ,num_results=10,verbose =True , tool_kwargs={"num_results": 10} )

# Print the search results
print(search_results)

This weekend Joel Engelberth shared a brand new interview with the G.O.A.T., Stern Pinball designer Keith Elwin on his YouTube show Flip N Out with Friends. In the interview, Elwin talks a lot about the design of his last two games, Godzilla and James Bond 60th, toppers and we even get a few tidbits about his next games - such as how his design for his next pinball machine has been done for ... Stern Pinball's JAWS Pro Edition (pictured) features Insider Connected as do the Premium and Limited Edition versions. Making its debut at CES 2024 in Las Vegas from Jan. 9-12, the latest game from Stern is officially in the world and ready to delight fans of the JAWS franchise and pinball in general. Designed by the factory's Keith Elwin ... The induced adrenaline rush wasn't based on us just playing 'another Stern pinball machine', we immediately noticed that Iron Maiden was a fresh take on the Stern formula with a new layout, thanks to Stern's rookie design team; Keith Elwin - Game Designer, 

In [23]:
search_results

"This weekend Joel Engelberth shared a brand new interview with the G.O.A.T., Stern Pinball designer Keith Elwin on his YouTube show Flip N Out with Friends. In the interview, Elwin talks a lot about the design of his last two games, Godzilla and James Bond 60th, toppers and we even get a few tidbits about his next games - such as how his design for his next pinball machine has been done for ... Stern Pinball's JAWS Pro Edition (pictured) features Insider Connected as do the Premium and Limited Edition versions. Making its debut at CES 2024 in Las Vegas from Jan. 9-12, the latest game from Stern is officially in the world and ready to delight fans of the JAWS franchise and pinball in general. Designed by the factory's Keith Elwin ... The induced adrenaline rush wasn't based on us just playing 'another Stern pinball machine', we immediately noticed that Iron Maiden was a fresh take on the Stern formula with a new layout, thanks to Stern's rookie design team; Keith Elwin - Game Designer,

In [29]:
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
search_query  = "keith elwin pinball designer"
search_results = search.run(  search_query ,max_results=10,verbose =True , tool_kwargs={"num_results": 10} )
search_results

[snippet: New Interview With Stern Pinball's Keith Elwin. This weekend Joel Engelberth shared a brand new interview with the G.O.A.T., Stern Pinball designer Keith Elwin on his YouTube show Flip N Out with Friends. In the interview, Elwin talks a lot about the design of his last two games, Godzilla and James Bond 60th, toppers and we even get a few ..., title: New Interview With Stern Pinball's Keith Elwin, link: https://www.knapparcade.org/post/new-interview-with-stern-pinball-s-keith-elwin], [snippet: Stern Pinball's JAWS Pro Edition (pictured) features Insider Connected as do the Premium and Limited Edition versions. Making its debut at CES 2024 in Las Vegas from Jan. 9-12, the latest game from Stern is officially in the world and ready to delight fans of the JAWS franchise and pinball in general. Designed by the factory's Keith Elwin ..., title: Spotlight Feature - Stern Pinball's JAWS, link: https://www.replaymag.com/stern-pinball-jaws-0224/], [snippet: Photo: Stern Pinball, Inc. 

"[snippet: New Interview With Stern Pinball's Keith Elwin. This weekend Joel Engelberth shared a brand new interview with the G.O.A.T., Stern Pinball designer Keith Elwin on his YouTube show Flip N Out with Friends. In the interview, Elwin talks a lot about the design of his last two games, Godzilla and James Bond 60th, toppers and we even get a few ..., title: New Interview With Stern Pinball's Keith Elwin, link: https://www.knapparcade.org/post/new-interview-with-stern-pinball-s-keith-elwin], [snippet: Stern Pinball's JAWS Pro Edition (pictured) features Insider Connected as do the Premium and Limited Edition versions. Making its debut at CES 2024 in Las Vegas from Jan. 9-12, the latest game from Stern is officially in the world and ready to delight fans of the JAWS franchise and pinball in general. Designed by the factory's Keith Elwin ..., title: Spotlight Feature - Stern Pinball's JAWS, link: https://www.replaymag.com/stern-pinball-jaws-0224/], [snippet: Photo: Stern Pinball, Inc.

In [ ]:
for x in search_results.__dir__():
  print(x)

In [40]:
import json
eval(search_results)

SyntaxError: invalid syntax (<string>, line 1)

### Results
You can see that even with the default settings we return 27 text snippets about the great Keith and some of the documents contain a decent amount of text. This makes our search API especially powerful for LLMs operating in the RAG-QA setting. But don't take my word for it, let's try it out on Hotpot QA.

### Hotpot QA
Let's take a look at an example from Hotpot. We load this up from the [Huggingface dataset](https://huggingface.co/datasets/hotpot_qa) using the datasets library. We use the fullwiki setting here instead of the distractor but as we said before, we'll be using our own context powered by the search APIs instead of what comes off the shelf.

In [12]:
from datasets import load_dataset


hotpot_ds = load_dataset("hotpot_qa", "fullwiki")["train"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpot_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [24]:
hotpot_ds[0]

{'id': '5a7a06935542990198eaf050',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'answer': "Arthur's Magazine",
 'type': 'comparison',
 'level': 'medium',
 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'],
  'sent_id': [0, 0]},
 'context': {'title': ['Radio City (Indian radio station)',
   'History of Albanian football',
   'Echosmith',
   "Women's colleges in the Southern United States",
   'First Arthur County Courthouse and Jail',
   "Arthur's Magazine",
   '2014–15 Ukrainian Hockey Championship',
   'First for Women',
   'Freeway Complex Fire',
   'William Rast'],
  'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launch

In [25]:
hotpot_ds[0]["question"]

"Which magazine was started first Arthur's Magazine or First for Women?"

The first question is asking about 2 magazines Arthur's Magazine and First for Women, specifically which was started first. I have never heard of either of these and indeed Hotpot is chock full of extremely niche questions which require knowledge across a large swath of time. Let's look at the context.

In [26]:
hotpot_ds[0]["context"]["sentences"]

[["Radio City is India's first private FM radio station and was started on 3 July 2001.",
  ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
  ' It plays Hindi, English and regional songs.',
  ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.',
  ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.',
  ' The Radio station currently plays a mix of Hindi and Regional music.',
  ' Abraham Thomas is the CEO of the company.'],
 ['Football in Albania existed before the Albanian Football Federation (FSHF) was created.',
  " This was evidenced by the team's registration at the Balkan Cup tournament during 1929-1931, which started in 1929 (although Albania eventually had

From the sentences we can see "Arthur's Magazine (1844–1846)" which we can assume means Arthur's ran from 1844-1846 while elsewhere it is mentioned "First for Women is a woman's magazine published by Bauer Media Group in the USA. The magazine was started in 1989." which means Arthur's came out first. Sure enough that is the answer.

In [27]:
hotpot_ds[0]["answer"]

"Arthur's Magazine"

Now let's use our `YouRetriever` in a RetrievalQA chain to see if we can answer this question using the the You.com Search API and GPT 3.5 Turbo.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


model = "gpt-3.5-turbo-16k"
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model=model), chain_type="stuff", retriever=yr)

In [ ]:
qa.run(hotpot_ds[0]["question"])

'Arthur magazine was started first. It was founded in October 2002. First for Women magazine was first published in 1989.'

We got it! Hurray! A quick note here is that we are using the 16k context window with GPT 3.5 Turbo because our API returns so much text by default it can overwhelm models with smaller context windows. Let's see what happens if we try the same thing with normal GPT 3.5 Turbo.

In [ ]:
from openai.error import InvalidRequestError


try:
    small_context_model = "gpt-3.5-turbo"
    small_context_qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model=small_context_model), chain_type="stuff", retriever=yr)
    small_context_qa.run(hotpot_ds[0]["question"])
except InvalidRequestError:
    print("Boom! Too much text!")

There are a few options you can employ here if you don't want to use a smaller context window model. The first is the cap the number of documents you feed from our API to the LLM. The other option is to use the [map_reduce chain](https://python.langchain.com/docs/modules/chains/document/map_reduce) type. The map_reduce chain type takes larger chunks of text and breaks them down to make them digestible by the LLM. This does mean that you will need to make multiple calls to the LLM which will mean slower run-time but you'll be able to process all the data returned from the `YouRetriever`.

In [ ]:
mr_small_context_qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model=small_context_model), chain_type="map_reduce", retriever=yr)
mr_small_context_qa.run(hotpot_ds[0]["question"])

"Based on the information available, it seems that Arthur's Magazine was started first. The text does not mention the beginning date of First for Women."

The chain is able to run but we can no longer answer the question without the full payload of documents in-context at one time it would seem. This is what makes large context window models so exciting LLMs are becoming extremely good at using a ton of text to answer questions.

### Head-to-Head Evaluation
Let's take a sample from Hotpot QA and compare our search API with one of the current alternatives in LangChain, the `GoogleSearchAPIWrapper`. This isn't a retriever in LangChain but it only takes a small amount of code to make an analog retriever. All we need to do is implement the `_get_relevant_documents` method of the abstract base class `BaseRetriever`. We should note here, that you could easily repeat this experiment and swap in another web search API like Bing. First let's create a small utility for the existing wrapper.

In [ ]:
from langchain.utilities import GoogleSearchAPIWrapper


search = GoogleSearchAPIWrapper()

def top10_results(query):
    return search.results(query, 10)

Now implement the `GoogleRetriever`.

In [ ]:
from langchain.schema.retriever import BaseRetriever, Document
from typing import TYPE_CHECKING, Any, Dict, List, Optional
from langchain.callbacks.manager import CallbackManagerForRetrieverRun, AsyncCallbackManagerForRetrieverRun


class GoogleRetriever(BaseRetriever):
    def __int__(self):
        pass

    def _get_relevant_documents(
            self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        return [Document(page_content=result.get("snippet", "")) for result in top10_results(query)]

    async def _aget_relevant_documents(
            self,
            query: str,
            *,
            run_manager: AsyncCallbackManagerForRetrieverRun,
            **kwargs: Any,
    ) -> List[Document]:
        raise NotImplementedError()

First, let's see the results of our new retriever on our test query.

In [ ]:
gr = GoogleRetriever()
gr.get_relevant_documents("keith elwin pinball designer")

[Document(page_content='Objectively, this may be the best pinball machine design of all time, including being better than AIQ. But my personal subjective preference for AIQ trumps it\xa0...'),
 Document(page_content='List of Keith Elwin Pinball Machines · JAWS · James Bond 007 60th Anniversary (LE) · Godzilla · Avengers: Infinity Quest · Jurassic Park · Iron Maiden: Legacy\xa0...'),
 Document(page_content='Keith Elwin was a member of the design team for the following pinball machines (ordered by date in descending order). Wishlist Collection Find on market · Jaws\xa0...'),
 Document(page_content='Keith Elwin is a professional pinball player turned designer from California. He started as an operator and technician and made a name for himself as an\xa0...'),
 Document(page_content='Aug 10, 2021 ... As I understand it Keith Elwin, Stern\'s hottest designer was brought on in the same way. He made an "Archer" homebrew that was picked up and\xa0...'),
 Document(page_content="Jan 13, 2024 ...

As you can see Google gives us much less information to feed into our LLM. While in both cases we requested results from 10 web results, the You.com Search API will attempt to give multiple text snippets per web result. To further demonstrate, we can now get predictions from the exact same LLM so we do our best to isolate the experiment to evaluating the search APIs.

In [ ]:
google_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model=model), chain_type="stuff", retriever=gr
)

Let's get a sample from our dataset.

In [ ]:
SAMPLE_SIZE = 20
hotpot_pds = hotpot_ds.to_pandas()
hotpot_pds_sample = hotpot_pds.sample(SAMPLE_SIZE, random_state=123).reset_index()

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


# This function is a simple way to parallelize calls to OpenAI in our pandas apply
def parallel_progress_apply(column, callback, num_workers):
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        return list(tqdm(executor.map(callback, column), total=len(column)))


# This function is a utility for our parallel pandas apply
def get_run_chain_function(chain):
    def run_chain(example):
        try:
            return chain(example)["result"]
        except:
            return ""
    return run_chain

Now we get predictions from the LLM using each search API's results. It's important to remember that at this point we have done everything we can to ensure the only thing we're testing is the quality of search results for use by an LLM to answer these questions.

In [ ]:
hotpot_pds_sample["ydc_prediction"] = parallel_progress_apply(
    hotpot_pds_sample["question"], lambda x: get_run_chain_function(qa)(x), num_workers=4
)
hotpot_pds_sample["google_prediction"] = hotpot_pds_sample["question"].apply(get_run_chain_function(google_qa))

100%|██████████| 20/20 [00:10<00:00,  1.88it/s]


### Calculating Scores
We use the F1 score function from the hotpot repository to ensure we are as close to the evaluation setting that was presented in the paper.

In [ ]:
import re
import string
from collections import Counter


# This is all taken from hotpot_qa source code with minor modifications to only return the f1 instead of the (P,R,F1) tuple
# https://github.com/hotpotqa/hotpot/blob/master/hotpot_evaluate_v1.py#L26
def calculate_f1_score(prediction, ground_truth):
    normalized_prediction = normalize_answer(prediction)
    normalized_ground_truth = normalize_answer(ground_truth)

    ZERO_METRIC = 0

    if (
        normalized_prediction in ["yes", "no", "noanswer"]
        and normalized_prediction != normalized_ground_truth
    ):
        return ZERO_METRIC
    if (
        normalized_ground_truth in ["yes", "no", "noanswer"]
        and normalized_prediction != normalized_ground_truth
    ):
        return ZERO_METRIC

    prediction_tokens = normalized_prediction.split()
    ground_truth_tokens = normalized_ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def filter_wiki_citation(snip):
    return not snip.startswith("- ^")

In [ ]:
hotpot_pds_sample["ydc_f1"] = parallel_progress_apply(
    list(hotpot_pds_sample.iterrows()),
    lambda x: calculate_f1_score(x[1]["ydc_prediction"], x[1]["answer"]),
    num_workers=8,
)
hotpot_pds_sample["google_f1"] = parallel_progress_apply(
    list(hotpot_pds_sample.iterrows()),
    lambda x: calculate_f1_score(x[1]["google_prediction"], x[1]["answer"]),
    num_workers=8,
)

100%|██████████| 20/20 [00:00<00:00, 331565.53it/s]


In [ ]:
print("You.com F1")
print(hotpot_pds_sample["ydc_f1"].mean())
print("Google F1")
print(hotpot_pds_sample["google_f1"].mean())

You.com F1
0.07386316842034746
Google F1
0.052014411027568916


### In Conclusion
As you can see, the You.com Search API heavily out-performs Google on this small subset of data. Please stay tuned as You.com will be releasing a much larger search study in the weeks to come. If you would like to be an early access partner of ours please email api@you.com with your background, use case, and expected daily call volume.